### NOTE!

Be sure to read the README as part of this Github repo to do the pre-requesite steps in preparing your AWS for 
usage by this script.


In [51]:
import os
import boto3
from botocore.client import ClientError
import tscribe

'''User Entry:

Please insert your AWS access key id and AWS secret access key.

NOTE: Do NOT push AWS secret access key to Git EVER!
'''
AWS_ACCESS_KEY_ID = "Enter in AWS access key id"
AWS_SECRET_ACCESS_KEY = "Enter in AWS secret access key"
'''
User Entry Area completed.
'''

session = boto3.Session( aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

# Part 1 - Creating Interview Transcriptions

Let's create the transcription job(s) for your audio/video interview files. The following cell will guide you in creating a proper request to AWS for your interview files. 

In [53]:
'''
User Entry:

Here is where you insert some specific AWS info
'''
S3_INTERVIEW_BUCKET_NAME = ""  # Name of s3 bucket where your interview files are
S3_TRANSCRIPT_BUCKET_NAME = "" # Name of s3 bucket where your transcript files will be put
'''
User Entry Area completed.
'''

s3 = session.resource('s3')

try:
    s3.meta.client.head_bucket(Bucket=S3_INTERVIEW_BUCKET_NAME)
except ClientError:
    # The bucket does not exist or you have no access.
    print(f"ClientError when trying to retreive: {S3_INTERVIEW_BUCKET_NAME}. The bucket does not exist or you have no access.\n")
    
try:
    s3.meta.client.head_bucket(Bucket=S3_TRANSCRIPT_BUCKET_NAME)
except ClientError:
    # The bucket does not exist or you have no access.
    print(f"ClientError when trying to retreive: {S3_TRANSCRIPT_BUCKET_NAME}. The bucket does not exist or you have no access.\n")


media_dict = {}
my_bucket = s3.Bucket(S3_INTERVIEW_BUCKET_NAME)
for my_bucket_object in my_bucket.objects.all():
    media_dict[my_bucket_object.key] = f"https://{S3_INTERVIEW_BUCKET_NAME}.s3.amazonaws.com/{my_bucket_object.key}"
    
print(media_dict)

ClientError when trying to retreive: infoorginterviewbucket. The bucket does not exist or you have no access
.
ClientError when trying to retreive: infororgtranscriptbucket. The bucket does not exist or you have no access.



ClientError: An error occurred (InvalidAccessKeyId) when calling the ListObjects operation: The AWS Access Key Id you provided does not exist in our records.

### WARNING!


The following cell instructs AWS to create transcriptions of your audio files! This means real requests are being made to AWS! As this can cost real resources be careful using this cell!

In [55]:
# job_uri = "s3 bucket link" + audio_file_name
# job_name = (audio_file_name.split('.')[0]).replace(" ", "")


transcribe = boto3.client('transcribe', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, 
                          region_name="us-east-1")

job_uri = ''
job_name = (''.split('.')[0]).replace(" ", "")

# job_name = check_job_name(job_name)

transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': job_uri},
    MediaFormat=''.split('.')[1],
    LanguageCode='en-US',
    Settings={'ShowSpeakerLabels': True,'MaxSpeakerLabels': 2},
    OutputBucketName=S3_TRANSCRIPT_BUCKET_NAME
)


ParamValidationError: Parameter validation failed:
Invalid length for parameter TranscriptionJobName, value: 0, valid range: 1-inf
Invalid length for parameter Media.MediaFileUri, value: 0, valid range: 1-inf